In [2]:
using Graphs
using SimpleWeightedGraphs
using CSV, DataFrames

In [3]:
network = "crossring/"
folder = network * "l3-c5-d2-bt2_test"
n_tsline = 3

3

In [4]:
TS_data = CSV.read( network * "crossring-network.csv", DataFrame, header = true)

Row,x,y,line,transfer
,Float64,Float64,Int64,String7?
1,-6.0,0.0,1,missing
2,-3.0,0.0,1,"11,19"
3,0.0,0.0,1,8
4,3.0,0.0,1,15
5,6.0,0.0,1,
6,0.0,6.0,2,
7,0.0,3.0,2,13
8,0.0,0.0,2,3
9,0.0,-3.0,2,17


In [5]:
# Function to parse strings into appropriate data types
function parse_column(str)
    if ismissing(str) || isempty(str)
        return Int[]
    elseif occursin(',', str)
        return parse.(Int, split(str, ','))
    else
        return [parse(Int, str)]
    end
end

TS_data.transfer = parse_column.(TS_data.transfer)

19-element Vector{Vector{Int64}}:
 []
 [11, 19]
 [8]
 [15]
 []
 []
 [13]
 [3]
 [17]
 []
 [2]
 []
 [7]
 []
 [4]
 []
 [9]
 []
 [2]

In [6]:
timetables = [CSV.read("$folder/timetable_line$i.csv", DataFrame) for i in 1:n_tsline]

3-element Vector{DataFrame}:
 3×6 DataFrame
 Row │ 1        2        3        4        5        Direction 
     │ Float64  Float64  Float64  Float64  Float64  Float64   
─────┼────────────────────────────────────────────────────────
   1 │    20.0     26.0     32.0     38.0     44.0        1.0
   2 │    49.0     43.0     37.0     31.0     25.0        0.0
   3 │    50.0     56.0     62.0     68.0     74.0        1.0
 2×6 DataFrame
 Row │ 6        7        8        9        10       Direction 
     │ Float64  Float64  Float64  Float64  Float64  Float64   
─────┼────────────────────────────────────────────────────────
   1 │    23.0     29.0     35.0     41.0     47.0        1.0
   2 │    52.0     46.0     40.0     34.0     28.0        0.0
 2×10 DataFrame
 Row │ 11       12       13       14       15       16       17       18       ⋯
     │ Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  ⋯
─────┼─────────────────────────────────────────────────────────────────────

In [7]:
n_layer = maximum(nrow.(timetables))
n_ts = size(TS_data)[1]

19

In [8]:
struct TransitStation
    ID::Int64 # the phsical number of transit stop
    coord::Tuple{Float64,Float64}
    layer_no::Int64
    dep_time::Float64
    arr_time::Float64
    lineNo::Int64 # at which transit line
    transfer::Vector{Int64}
    μ::Float64 # service time
end

In [9]:
# Base.isnan(ts::TransitStation) = !isa(ts, TransitStation)

In [10]:
ts_map = permutedims(reshape(1:n_ts*n_layer, n_ts, n_layer)) # row: layer number; column: transit stops

3×19 Matrix{Int64}:
  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
 20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38
 39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57

In [11]:
dwel_time = 1
ts_nodes = Vector(undef, n_ts*n_layer)
for lyr in 1:n_layer
    for ts in 1:n_ts
        coord = (TS_data[ts,:x], TS_data[ts,:y])
        lineNo = TS_data[ts,:line]
        transfer = TS_data[ts, :transfer]
        if lyr <= size(timetables[lineNo])[1] 
            dep_time = timetables[lineNo][lyr,"$ts"]
            arr_time = dep_time .- dwel_time
            ts_dum = ts_map[lyr, ts]
            ts_nodes[ts_dum] = TransitStation(ts, coord, lyr, dep_time, arr_time, lineNo, transfer, 0.0)
        else
            ts_dum = ts_map[lyr, ts]
            ts_map[lyr, ts] = 0
            ts_nodes[ts_dum] = NaN
        end
    end
end

In [12]:
# filter!(x -> isa(x,TransitStation), ts_nodes)
ts_nodes

57-element Vector{Any}:
    TransitStation(1, (-6.0, 0.0), 1, 20.0, 19.0, 1, Int64[], 0.0)
    TransitStation(2, (-3.0, 0.0), 1, 26.0, 25.0, 1, [11, 19], 0.0)
    TransitStation(3, (0.0, 0.0), 1, 32.0, 31.0, 1, [8], 0.0)
    TransitStation(4, (3.0, 0.0), 1, 38.0, 37.0, 1, [15], 0.0)
    TransitStation(5, (6.0, 0.0), 1, 44.0, 43.0, 1, Int64[], 0.0)
    TransitStation(6, (0.0, 6.0), 1, 23.0, 22.0, 2, Int64[], 0.0)
    TransitStation(7, (0.0, 3.0), 1, 29.0, 28.0, 2, [13], 0.0)
    TransitStation(8, (0.0, 0.0), 1, 35.0, 34.0, 2, [3], 0.0)
    TransitStation(9, (0.0, -3.0), 1, 41.0, 40.0, 2, [17], 0.0)
    TransitStation(10, (0.0, -6.0), 1, 47.0, 46.0, 2, Int64[], 0.0)
   ⋮
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

In [13]:
import Base.+
+(range::UnitRange{Int64}, offset::Int64) = UnitRange(range.start + offset, range.stop + offset)

+ (generic function with 218 methods)

In [14]:
stops_line = [count(x -> x == i, TS_data.line) for i in 1:n_tsline]
start_indices = cumsum([1;stops_line[1:end-1]])
ranges_list = [start: start + length - 1 for (start, length) in zip(start_indices, stops_line)]

3-element Vector{UnitRange{Int64}}:
 1:5
 6:10
 11:19

In [15]:
function find_transfer_arcs(ts1_dm::Int64, ts_nodes::Vector, ts_map::Matrix; max_transfer_time = 10.0)
    ts1 = ts_nodes[ts1_dm]
    transfers = ts1.transfer
    arcs = Vector{NamedTuple{(:ts1, :ts2, :tt), Tuple{Int64, Int64, Float64}}}()
    for ts2_ID in transfers
        ts2_dummies = ts_map[:,ts2_ID]
        filter!(x -> x != 0, ts2_dummies)
        for ts2_dm in ts2_dummies
            ts2 = ts_nodes[ts2_dm]
            tt = ts2.dep_time - ts1.arr_time
            if tt > 0 && tt <= max_transfer_time
                push!(arcs, (ts1 = ts1_dm, ts2 = ts2_dm, tt = tt))
            end
        end
    end
    return arcs
end

find_transfer_arcs (generic function with 1 method)

In [16]:
access_arcs = Vector{NamedTuple{(:ts1, :ts2, :tt), Tuple{Int64, Int64, Float64}}}()
transfer_arcs = Vector{NamedTuple{(:ts1, :ts2, :tt), Tuple{Int64, Int64, Float64}}}()
arcs_dict = Dict((l,lyr) =>[] for lyr in 1:n_layer for l in 1:n_ts)
max_transfer_time = 10.0
for lyr in 1:n_layer
    for l in 1:n_tsline
        ts_range = ranges_list[l] + 19*(lyr-1)
        if ts_map[lyr,ranges_list[l].start] == 0
            continue
        end
        direction = timetables[l].Direction[lyr]
        # println("ts range: ", ts_range)
        
        for ts1 in ts_range
            # arcs within the same line
            for ts2 in ts_range             
                if direction == 1 && ts2 > ts1
                    tt = ts_nodes[ts2].dep_time - ts_nodes[ts1].arr_time
                    push!(access_arcs, (ts1 = ts1, ts2 = ts2, tt = tt))
                    push!(arcs_dict[(l,lyr)], (ts1,ts2))
                elseif direction == 0 && ts1 < ts2
                    tt = ts_nodes[ts1].dep_time - ts_nodes[ts2].arr_time
                    push!(access_arcs, (ts1 = ts2, ts2 = ts1, tt = tt))
                    push!(arcs_dict[(l,lyr)], (ts2, ts1))
                end
            end
            
            # trasfer arcs to different lines
            if isempty(ts_nodes[ts1].transfer)
                continue
            else
                transfer_arcs_node = find_transfer_arcs(ts1, ts_nodes, ts_map, max_transfer_time = max_transfer_time)
                if !isempty(transfer_arcs_node)
                    println("connect: ", transfer_arcs_node)
                    # access_arcs = [access_arcs; transfer_arcs]
                    transfer_arcs = [transfer_arcs; transfer_arcs_node]
                end
            end
        end

    end
end

connect: 

NamedTuple{(:ts1, :ts2, :tt), Tuple{Int64, Int64, Float64}}[(ts1 = 2, ts2 = 11, tt = 1.0), (ts1 = 2, ts2 = 38, tt = 6.0)]
connect: NamedTuple{(:ts1, :ts2, :tt), Tuple{Int64, Int64, Float64}}[(ts1 = 3, ts2 = 8, tt = 4.0), (ts1 = 3, ts2 = 27, tt = 9.0)]
connect: NamedTuple{(:ts1, :ts2, :tt), Tuple{Int64, Int64, Float64}}[(ts1 = 7, ts2 = 13, tt = 10.0)]
connect: NamedTuple{(:ts1, :ts2, :tt), Tuple{Int64, Int64, Float64}}[(ts1 = 8, ts2 = 22, tt = 3.0)]
connect: NamedTuple{(:ts1, :ts2, :tt), Tuple{Int64, Int64, Float64}}[(ts1 = 9, ts2 = 36, tt = 3.0)]
connect: NamedTuple{(:ts1, :ts2, :tt), Tuple{Int64, Int64, Float64}}[(ts1 = 11, ts2 = 2, tt = 1.0)]
connect: NamedTuple{(:ts1, :ts2, :tt), Tuple{Int64, Int64, Float64}}[(ts1 = 13, ts2 = 26, tt = 9.0)]
connect: NamedTuple{(:ts1, :ts2, :tt), Tuple{Int64, Int64, Float64}}[(ts1 = 

22, ts2 = 27, tt = 4.0)]
connect: NamedTuple{(:ts1, :ts2, :tt), Tuple{Int64, Int64, Float64}}[(ts1 = 28, ts2 = 36, tt = 10.0)]


In [56]:
length(access_arcs)
length(transfer_arcs)

11

In [17]:
ts_graph = SimpleWeightedDiGraph(n_ts*n_layer)

{57, 0} directed simple Int64 graph with Float64 weights

In [18]:
for arc in access_arcs
    add_edge!(ts_graph, arc.ts1, arc.ts2, arc.tt)
end
for arc in transfer_arcs
    add_edge!(ts_graph, arc.ts1, arc.ts2, arc.tt)
end
ts_graph

{57, 133} directed simple Int64 graph with Float64 weights

In [19]:
function shortest_ts_arc!(nodes_o::Int64, traveltime_ts::Matrix{Float64})
    r = dijkstra_shortest_paths(ts_graph, node_o)
    active_arcs = []
    fixed = []
    for (i,tt) in enumerate(r.dists)
        identity_arc = Dict()
        if isinf(tt) || tt <= 0
            continue
        else
            if (ts_nodes[i].ID ∉ fixed) && (ts_nodes[i].ID != node_o)
                if isempty(ts_nodes[i].transfer)
                    push!(fixed, ts_nodes[i].ID)
                    push!(active_arcs,(node_o, i, tt))
                    traveltime_ts[nodes_o,i] = tt
                    # println("($node_o, $i): is $tt minutes")
                else    
                    # keep the shortest travel time (node_o,i) and (node_o,tfr)
                    compares = [i; ts_nodes[i].transfer]
                    shortes_ts = argmin(r.dists[compares])
                    shortes_ts = compares[shortes_ts]    
                    fixed = [fixed; compares]
                    push!(active_arcs, (node_o, i, tt)) 
                    traveltime_ts[nodes_o, i] = tt
                    # println("($node_o, $i): is $tt minutes")       
                end
            else
                continue
            end
            
        end
    end
    return active_arcs
end

shortest_ts_arc! (generic function with 1 method)

In [60]:
traveltime_ts = 1000 * ones(maximum(ts_map), maximum(ts_map))
node_o = 2
active_arcs = shortest_ts_arc!(2, traveltime_ts)

14-element Vector{Any}:
 (2, 3, 7.0)
 (2, 4, 13.0)
 (2, 5, 19.0)
 (2, 9, 18.0)
 (2, 10, 24.0)
 (2, 11, 1.0)
 (2, 12, 8.0)
 (2, 13, 14.0)
 (2, 14, 20.0)
 (2, 16, 32.0)
 (2, 18, 44.0)
 (2, 19, 50.0)
 (2, 20, 27.0)
 (2, 25, 29.0)

In [20]:
get_arcs(arc::NamedTuple{(:ts1, :ts2, :tt), Tuple{Int64, Int64, Float64}}) = (arc.ts1,arc.ts2)

get_arcs (generic function with 1 method)

In [41]:
traveltime_ts = Inf * ones(n_ts*n_layer, n_ts*n_layer)
traveltime_ts_nowaiting = Inf * ones(n_ts*n_layer, n_ts*n_layer)
paths_ts = Dict{Tuple, Vector}()
arcs_transfer = get_arcs.(transfer_arcs)
for node_o in 1:n_ts*n_layer
    r = dijkstra_shortest_paths(ts_graph, node_o)
    traveltime_ts[node_o,:] = r.dists
    for node_d in 1:n_ts*n_layer
        path = enumerate_paths(r,node_d)
        if node_o == node_d
            traveltime_ts[node_o, node_d] == Inf
        elseif (node_o, node_d) ∈ arcs_transfer
            traveltime_ts[node_o, node_d] == Inf
        elseif length(path) < 2
            continue
        elseif length(path) == 3
            traveltime_ts[node_o, node_d] == Inf
            # println(node_o, ", ", node_d)
        else
            paths_ts[(node_o, node_d)] = path
        end
    end
end

In [42]:
ts_map

3×19 Matrix{Int64}:
  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
 20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38
 39  40  41  42  43   0   0   0   0   0   0   0   0   0   0   0   0   0   0

In [43]:
for i in 1:n_ts*n_layer
# for i in 1:1
    for j in 1:n_ts*n_layer
    # for j in 2:2
        if isa(ts_nodes[j], TransitStation)
            if isempty(ts_nodes[j].transfer)
                continue
            else
                # @info j
                ts_phy = findfirst(x->x==j,ts_map)[2]
                compares = hcat(ts_map[:,[ts_phy;ts_nodes[j].transfer]]...)[1,:]
                filter!(x->x!=0, compares)
                # @info compares
                select = compares[argmin(traveltime_ts[i,compares])]
                if (i == 28) && (j == 35)
                    @info select
                end
                filter!(x->x!=select,compares)
                # @info select, compares
                traveltime_ts[i,compares] .= Inf
                for arc in [(i,k) for k in compares]
                    delete!(paths_ts, arc)
                end
            end
        end
    end
end

In [44]:
paths_ts

Dict{Tuple, Vector} with 180 entries:
  (32, 30) => [32, 30]
  (7, 8)   => [7, 8]
  (14, 15) => [14, 15]
  (1, 9)   => [1, 3, 8, 9]
  (38, 33) => [38, 33]
  (27, 26) => [27, 26]
  (28, 25) => [28, 25]
  (17, 19) => [17, 19]
  (16, 18) => [16, 18]
  (3, 33)  => [3, 8, 9, 36, 33]
  (23, 21) => [23, 21]
  (1, 3)   => [1, 3]
  (7, 25)  => [7, 13, 26, 25]
  (1, 20)  => [1, 3, 8, 22, 20]
  (15, 19) => [15, 19]
  (22, 20) => [22, 20]
  (3, 4)   => [3, 4]
  (6, 31)  => [6, 9, 36, 31]
  (36, 35) => [36, 35]
  ⋮        => ⋮

In [33]:
paths_ts[(28,35)]
paths_ts[(36,35)]
traveltime_ts[36,35]

7.0

In [143]:
compares
to_delet = [(1,k) for k in compares]
for arc in [(1,k) for k in compares]
    delete!(paths_ts, arc)
end

In [144]:
paths_ts[(1,21)]

KeyError: KeyError: key (1, 21) not found

In [63]:
for arc in transfer_arcs
    println(arc)
end

(ts1 = 2, ts2 = 11, tt = 1.0)
(ts1 = 2, ts2 = 38, tt = 6.0)
(ts1 = 3, ts2 = 8, tt = 4.0)
(ts1 = 3, ts2 = 27, tt = 9.0)
(ts1 = 7, ts2 = 13, tt = 10.0)
(ts1 = 8, ts2 = 22, tt = 3.0)
(ts1 = 9, ts2 = 36, tt = 3.0)
(ts1 = 11, ts2 = 2, tt = 1.0)
(ts1 = 13, ts2 = 26, tt = 9.0)
(ts1 = 22, ts2 = 27, tt = 4.0)
(ts1 = 28, ts2 = 36, tt = 10.0)


In [101]:
for i in 1:n_ts*n_layer    
    for arc in transfer_arcs
        ts1, ts2 = arc.ts1, arc.ts2
        if isinf(traveltime_ts[i,ts1]) || isinf(traveltime_ts[i,ts2])
            continue
        elseif traveltime_ts[i,ts1] <= traveltime_ts[i,ts2]
            traveltime_ts[i,ts2] == Inf
            delete!(paths_ts, (i,ts2))
        elseif traveltime_ts[i,ts1] > traveltime_ts[i,ts2]
            traveltime_ts[i, ts1] == Inf
            delete!(paths_ts, (i,ts1))
        end
    end
end

In [103]:
ts_arcs = keys(paths_ts)

KeySet for a Dict{Tuple, Vector} with 210 entries. Keys:
  (11, 17)
  (32, 30)
  (7, 8)
  (14, 15)
  (1, 9)
  (38, 33)
  (27, 26)
  (28, 25)
  (17, 19)
  (16, 18)
  (11, 32)
  (3, 33)
  (23, 21)
  (1, 3)
  (7, 25)
  (1, 20)
  (15, 19)
  (22, 20)
  (3, 4)
  ⋮

In [110]:
paths_ts[(11,9)]

5-element Vector{Int64}:
 11
  2
  3
  8
  9

In [111]:
paths_ts[(11,17)]

2-element Vector{Int64}:
 11
 17

In [86]:
ts_arcs = []
for i in 1:n_ts*n_layer
    for j in 1:n_ts*n_layer
        if traveltime_ts[i,j] > 0 && !isinf(traveltime_ts[i,j])
            push!(ts_arcs, (i,j))
        end
    end
end
ts_arcs

290-element Vector{Any}:
 (1, 2)
 (1, 3)
 (1, 4)
 (1, 5)
 (1, 8)
 (1, 9)
 (1, 10)
 (1, 11)
 (1, 12)
 (1, 13)
 ⋮
 (39, 41)
 (39, 42)
 (39, 43)
 (40, 41)
 (40, 42)
 (40, 43)
 (41, 42)
 (41, 43)
 (42, 43)

In [89]:
paths_ts[(1,15)]

4-element Vector{Int64}:
  1
  2
 11
 15

In [178]:
enumerate_paths(r,30)

2-element Vector{Int64}:
 37
 30

In [138]:
TS_data.transfer

19-element Vector{Vector{Int64}}:
 []
 [11, 19]
 [8]
 [15]
 []
 []
 [13]
 [3]
 [17]
 []
 [2]
 []
 [7]
 []
 [4]
 []
 [9]
 []
 [2]

In [153]:
trans_group = Dict()
for (i,v) in enumerate(TS_data.transfer)
    if !isempty(v)
        if !isempty(intersect(TS_data.transfer[i], keys(trans_group)))
            continue
        end
        trans_group[i] = [i;v]
    end
end

In [161]:
sort(trans_group)

OrderedCollections.OrderedDict{Any, Any} with 5 entries:
  2 => [2, 11, 19]
  3 => [3, 8]
  4 => [4, 15]
  7 => [7, 13]
  9 => [9, 17]

In [160]:
trans_arcs = Dict(k => [] for k in keys(trans_group))

Dict{Int64, Vector{Any}} with 5 entries:
  4 => []
  7 => []
  2 => []
  9 => []
  3 => []

In [163]:
for (k,v) in trans_group
    for ts in v
        if r.dists[v]
        end
    end
end

57-element Vector{Float64}:
  0.0
  7.0
 13.0
 19.0
 25.0
 Inf
 Inf
 17.0
 24.0
 30.0
  ⋮
 Inf
 Inf
 Inf
 Inf
 Inf
 Inf
 Inf
 Inf
 Inf